In [1]:
import pandas as pd
import psycopg2
import config
import workdays
import datetime


holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')
dt = datetime.date.today()
vcto_0 = dt.strftime('%d/%m/%Y')
dt_pos = workdays.workday(dt, -1, holidays_br)
dt_1 = workdays.workday(dt, -1, holidays_b3)

db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)
# cursor_test = db_conn_test.cursor(cursor_factory=psycopg2.extras.DictCursor)

db_conn_risk = psycopg2.connect(host=config.DB_RISK_HOST, dbname=config.DB_RISK_NAME , user=config.DB_RISK_USER, password=config.DB_RISK_PASS)
# cursor_risk = db_conn_risk.cursor(cursor_factory=psycopg2.extras.DictCursor)

db_conn_kapv1 = psycopg2.connect(host=config.DB_KAPV1_HOST, dbname=config.DB_KAPV1_NAME , user=config.DB_KAPV1_USER, password=config.DB_KAPV1_PASS)



In [132]:
def get_offshore():
    query="SELECT str_codigo,str_serie,dbl_qtetrade,dbl_putrade \
            FROM public.tbl_boletasexternas where (str_codigo='ES' or str_codigo='MES@CME')  and str_mesa='Kapitalo 11.1' and dte_data>= '2021-11-01'"
    df=pd.read_sql(query,db_conn_test)
    
    return df

def get_onshore():
    
    query = "SELECT str_codigo,str_serie,dbl_lote,dbl_preco \
FROM public.tbl_auxboletas1 where (str_codigo='ISP' or str_codigo='WSP' or str_codigo='RSP') and (str_serie= 'H22' or str_serie='Z1H2') and str_mesa='Kapitalo 11.1' and dte_data>= '2021-11-01'"
    df=pd.read_sql(query,db_conn_test)
    
    return df

df_2=get_onshore()



df_2=df_2.rename(columns={'dbl_lote':'dbl_qtetrade','dbl_preco':'dbl_putrade'})

df_1=get_offshore()

df=pd.concat([df_1,df_2],ignore_index=True)

ratios = {
    'ISP': 1,
    'ES': 1,
    'MES@CME': 1 / 10,
    'WSP': 1 / 20
}

parent = {
    'ISP': 'ISP',
    'ES': 'ES',
    'MES@CME': 'ES',
    'WSP': 'ISP',
    
}

df['real_notional'] = df['str_codigo'].map(ratios) * df['dbl_qtetrade']
df['parent'] = df['str_codigo'].map(parent)
# df['side'] = df['dbl_qtetrade'].apply(lambda x: 'B' if x > 0 else 'S')
# df['dbl_qtetrade']=df.apply(lambda row: (row['dbl_qtetrade']/20) if row['str_codigo']=='WSP' else (row['dbl_qtetrade']/10) if row['dbl_qtetrade']=='MES@CME' else row['dbl_qtetrade'],axis=1)


# df['base_isp']= df['dbl_qtetrade']*df['dbl_putrade']

# df=df.groupby('str_codigo',as_index=False).agg({'dbl_qtetrade':sum,'base_isp':sum})



# # df['base_isp']=df.apply(lambda row: (row['total']/20) if row['str_codigo']=='WSP' else (row['total']/10) if row['str_codigo']=='MES@CME' else row['total'],axis=1)

# # df_on=df[[df['str_codigo']=='ISP' & df['str_codigo']=='WSP' ]]
# df.to_excel('isp.xlsx')


In [124]:
df['volume'] = df['real_notional'] * df['dbl_putrade']

In [125]:
df.groupby(['parent']).sum().assign(avg=lambda row: row['volume'] / row['real_notional'])

,dbl_qtetrade,dbl_putrade,real_notional,volume,avg
parent,,,,,
ES,681.0,9428856.25,276.9,1.283307e+06,4634.550921
ISP,-1575.0,2871183.00,-685.8,-3.225312e+06,4702.992290


-38.682566000000406

In [126]:
off=get_offshore()

off['real_notional'] = off['str_codigo'].map(ratios) * off['dbl_qtetrade']
off['volume'] = off['real_notional'] * off['dbl_putrade']
off['parent'] = off['str_codigo'].map(parent)

off.groupby(['parent','str_serie']).sum().assign(avg=lambda row: row['volume'] / row['real_notional'])





dbl_qtetrade  dbl_putrade  real_notional       volume  \
parent str_serie                                                          
ES     H22              4078.0   4793186.00         3656.8  17056447.80   
       Z21             -3397.0   4635670.25        -3379.9 -15773140.65   

                          avg  
parent str_serie               
ES     H22        4664.309724  
       Z21        4666.747729

In [155]:
df_rsp=df_2[df_2['str_codigo']=='RSP']
# df_rsp.groupby(['str_codigo','str_serie']).sum()

df_rsp['volume']=df_rsp['dbl_qtetrade']*df_rsp['dbl_putrade']

# df_rsp
df_rsp.groupby(['str_codigo','str_serie']).sum().assign(avg=lambda row: row['volume'] / row['dbl_qtetrade'])



C:\Users\JOAO~1.RAM\AppData\Local\Temp/ipykernel_4200/3959878926.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rsp['volume']=df_rsp['dbl_qtetrade']*df_rsp['dbl_putrade']


,,dbl_qtetrade,dbl_putrade,volume,avg
str_codigo,str_serie,,,,
RSP,Z1H2,-2971.0,24.5,-149.9,0.050454


In [158]:
4078-2971


1107